#### Funciones auxiliares

In [4]:
# Mostrar contenido del fichero
def showFile(fileName):
    with open(fileName, "r", encoding="utf-8") as file:
        contenido = file.read() 
        print(contenido)

showFile("test.grammar")

S->aA|bB
A->aA|a
B->bB|b


# Compilar fichero

Partimos de un fichero con la gramática en forma normal de Chomsky.

RECUERDA PONER EN LA PRIMERA LINEA EL SÍMBOLO GENERADOR

In [15]:
showFile("test.grammar")

S->aA|bB
A->aA|a
B->bB|b


Para compilar un fichero hay que llamar a la función `compileFile` de `compiler`, tras la llamada nos devolverá una objeto que contiene la red celular

In [16]:
from compiler import compileFile
cellNet = compileFile("test.grammar")
cellNet

== N1 ==

MUTATION
S->[aA]
S->[bB]
A->[aA]
A->a
B->[bB]
B->b

POPULATION:
S

[>>] INPUT FILTER:
excluded(['[^A]', '[^B]', '[^S]', '[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]'])

[<<] OUTPUT FILTER:
permited(['[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]'])

CONNECTIONS:
N2
N3

== N2 ==

CROSSOVER

POPULATION:
[^B]
[^A]
[^S]

[>>] INPUT FILTER:
permited(['[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]'])

[<<] OUTPUT FILTER:
(VV | permited(['A', 'B', 'S', '[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]', 'a', 'b']))

CONNECTIONS:
N1
N3

== N3 ==

MUTATION
[bA]->b
[aA]->a
[bB]->b
[aB]->a
[bS]->b
[aS]->a
[^A]->A
[^B]->B
[^S]->S

POPULATION:

[>>] INPUT FILTER:
{[aA]Â : a ∈ T ∧ A ∈ N}

[<<] OUTPUT FILTER:
excluded(['A', 'B', 'S', '[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]'])

CONNECTIONS:
N1
N2


# Compilar código en string

También se pueden compilar código escrito directamente en un string

In [2]:
from compiler import compileCode
code = """
S->aA|bB
A->aA|a
B->bB|b
"""
cellNet = compileCode(code)
cellNet

== N1 ==

MUTATION
S->[aA]
S->[bB]
A->[aA]
A->a
B->[bB]
B->b

POPULATION:
S

[>>] INPUT FILTER:
excluded(['[^A]', '[^B]', '[^S]', '[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]'])

[<<] OUTPUT FILTER:
permited(['[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]'])

CONNECTIONS:
N2
N3

== N2 ==

CROSSOVER

POPULATION:
[^B]
[^S]
[^A]

[>>] INPUT FILTER:
permited(['[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]'])

[<<] OUTPUT FILTER:
(VV | permited(['A', 'B', 'S', '[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]', 'a', 'b']))

CONNECTIONS:
N1
N3

== N3 ==

MUTATION
[bA]->b
[aA]->a
[bB]->b
[aB]->a
[bS]->b
[aS]->a
[^A]->A
[^B]->B
[^S]->S

POPULATION:

[>>] INPUT FILTER:
{[aA]Â : a ∈ T ∧ A ∈ N}

[<<] OUTPUT FILTER:
excluded(['A', 'B', 'S', '[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]'])

CONNECTIONS:
N1
N2


## Simular

El objeto devulto no solo representa a la red, también actua como simulador, usando el método run.

Se puede llamar con el atributo `ticks` y/o el atributo `seconds`, para determinar cuanto va a durar la simulación.

In [3]:
trace = cellNet.run(ticks=5)

El método devuelve la traza de todas las operaciones realizadas

In [5]:
print(trace)

N3: no operation: no population in this cell
N2: no move: no word can pass the output filter
N2: no move: no word can pass the output filter
N1: mutation: ['S'] -> ['[bB]']
N3: no operation: no population in this cell



# Consultar una célula

Las células se pueden consultar por su nombre, en este caso vamos. Usando el método getCell podemos objetener un objeto célula que podemos consultar y manipular.

In [8]:
N1 = cellNet.getCell("N1")
N1

== N1 ==

MUTATION
S->[aA]
S->[bB]
A->[aA]
A->a
B->[bB]
B->b

POPULATION:
[bB]

[>>] INPUT FILTER:
excluded(['[^A]', '[^B]', '[^S]', '[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]'])

[<<] OUTPUT FILTER:
permited(['[aA]', '[aB]', '[aS]', '[bA]', '[bB]', '[bS]'])

CONNECTIONS:
N2
N3

In [9]:
N1.population

[['[bB]']]

# Crear una red celular manualmente

En este ejemplo se muestra cómo crear una red celular desde cero, definiendo células, operaciones y conexiones.

In [11]:
from cell import Cell
from mutation import Mutation
from crossover import Crossover
from cell_net import CellNet
from filters import permited, excluded

# Definir poblaciones iniciales
pop1 = [["a", "b", "a"], ["b", "a", "b"]]
pop2 = [["c", "c", "a"], ["a", "c", "b"]]

# Definir reglas de mutación para la primera célula
rules1 = {
    "a": ["A"],
    "b": ["B"]
}

# Crear operaciones
mutation_op = Mutation(rules1, pop1)
crossover_op = Crossover(population=pop2)

# Definir filtros: solo permitir palabras que contengan al menos una 'a' y excluir palabras con 'c'
permited_filter = permited({'a'})
excluded_filter = excluded({'c'})

# Crear células con filtros personalizados
cell1 = Cell(name="C1", population=pop1, operation=mutation_op, inFilter=permited_filter, outFilter=excluded_filter)
cell2 = Cell(name="C2", population=pop2, operation=crossover_op, inFilter=permited_filter, outFilter=excluded_filter)

# Conectar las células entre sí
cell1.connections = [cell2]
cell2.connections = [cell1]

# Crear la red celular
my_cellNet = CellNet([cell1, cell2])

my_cellNet

== C1 ==

MUTATION
a->A
b->B

POPULATION:
aba
bab

[>>] INPUT FILTER:
permited(['a'])

[<<] OUTPUT FILTER:
excluded(['c'])

CONNECTIONS:
C2

== C2 ==

CROSSOVER

POPULATION:
cca
acb

[>>] INPUT FILTER:
permited(['a'])

[<<] OUTPUT FILTER:
excluded(['c'])

CONNECTIONS:
C1


In [13]:
# Al igual que las redes creadas anteriormente también se pueden simular
trace = my_cellNet.run(seconds=0.01)

In [14]:
my_cellNet

== C1 ==

MUTATION
a->A
b->B

POPULATION:
ABB
A
Aa
ab
AA
A
a
a

[>>] INPUT FILTER:
permited(['a'])

[<<] OUTPUT FILTER:
excluded(['c'])

CONNECTIONS:
C2

== C2 ==

CROSSOVER

POPULATION:
cca
acb
acb
cb
b
cacb
ccb
cb
ccacb
cccb
ccb
ccaacb
ccacb
ccab
cca
ca
acca
aca
accca
acca
aca
acbcca
acbca
acba
aBa
cb
b
acb
ab
accb
acb
acacb
acab
aca
ca
caca
cca
ca
cbaca
cbca
cba
accb
ccb
cb
b
aaccb
accb
acb
ab
aa
aca
acca
accba
ccacb
cacb
acb
cb
accacb
acacb
aacb
acb
ab
aBccacb
aBcacb
aBacb
aBcb
aBaccacb
aBacacb
aBaacb
aBacb
aBa
Ba
a
caBa
cBa
ca
ccaBa
ccBa
cca
ccaaBa
ccaBa
ccaa
ccacaBa
ccacBa
ccaca
ccacbaBa
ccacbBa
ccacba
acb
cb
b
cacb
ccb
cb
ccacb
cccb
ccb
ccaacb
ccacb
ccab
ccacacb
ccaccb
ccacb
ccacbacb
ccacbcb
ccacbb
ccacb
cacb
acb
cb
accacb
acacb
aacb
acb
ab
acccacb
accacb
acacb
accb
acb
acbccacb
acbcacb
acbacb
acbcb
acbb
accb
ccb
cb
caccb
cccb
ccb
cb
ccaccb
ccccb
cccb
ccb
ccaaccb
ccaccb
ccacb
ccab
ccacaccb
ccacccb
ccaccb
ccacb
ccacaaccb
ccacaccb
ccacacb
ccacab
ccacacaccb
ccacacccb
ccacaccb
ccaca

In [15]:
print(trace)

C2: crossover: ['c', 'c', 'a'] + ['a', 'c', 'b']
C1: mutation: ['b', 'a', 'b'] -> ['b', 'a', 'B']
C1: move: ['b', 'a', 'B'] entered in cell C2
C1: move: ['a', 'b', 'a'] entered in cell C2
C2: move: ['a', 'b', 'a'] entered in cell C1
C1: mutation: ['a', 'b', 'a'] -> ['a', 'B', 'a']
C1: move: ['a', 'B', 'a'] entered in cell C2
C2: move: ['a', 'a'] entered in cell C1
C2: crossover: ['a', 'c', 'a'] + ['c', 'b']
C2: crossover: ['a'] + ['a', 'c', 'c', 'b']
C1: mutation: ['a', 'a'] -> ['a', 'A']
C2: crossover: ['a', 'B', 'a'] + ['c', 'c', 'a', 'c', 'b']
C1: mutation: ['a', 'A'] -> ['A', 'A']
C1: no mutation: no posible replacement for the word ['A', 'A']
C2: crossover: ['c', 'c', 'a', 'c', 'b'] + ['a', 'c', 'b']
C1: no mutation: no posible replacement for the word ['A', 'A']
C1: no mutation: no posible replacement for the word ['A', 'A']
C2: move: ['b'] deleted in cell C1
C1: move: ['A', 'A'] deleted in cell C2
C2: crossover: ['c', 'c', 'a', 'c', 'a', 'c', 'b'] + ['a', 'c', 'c', 'b']
C1: no o